In [1]:
"""proof of concept example for deep neural network NB regression
Parts of the code are from Goekcen Eraslan
https://github.com/gokceneraslan/autoencoder
python 3 requirements:
----------------------
numpy
tensorflow>=1.0.0
keras>=2.0.0
matplotlib
"""

'proof of concept example for deep neural network NB regression\nParts of the code are from Goekcen Eraslan\nhttps://github.com/gokceneraslan/autoencoder\npython 3 requirements:\n----------------------\nnumpy\ntensorflow>=1.0.0\nkeras>=2.0.0\nmatplotlib\n'

In [3]:
import numpy as np
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model

In [5]:
from keras import backend as K
import os
import tensorflow as tf
from tensorflow.keras.layers import Layer
import matplotlib.pyplot as plt

In [6]:
class SliceLayer(Layer):
    def __init__(self, index, **kwargs):
        self.index = index
        super().__init__(**kwargs)

    def build(self, input_shape):
        if not isinstance(input_shape, list):
            raise ValueError('Input should be a list')

        super().build(input_shape)

    def call(self, x):
        return x[self.index]

    def compute_output_shape(self, input_shape):
        return input_shape[self.index]

In [12]:
class NB(object):
    def __init__(self, theta=None, scope='nbinom_loss/',
                 scale_factor=1.0, debug=False):

        # for numerical stability
        self.eps = 1e-10
        self.scale_factor = scale_factor
        self.debug = debug
        self.scope = scope
        self.theta = theta

    def loss(self, y_true, y_pred, reduce=True):
        scale_factor = self.scale_factor
        eps = self.eps

        with tf.name_scope(self.scope):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.cast(y_pred, tf.float32) * scale_factor

            # Clip theta
            theta = tf.minimum(self.theta, 1e6)

            t1 = -tf.math.lgamma(y_true+theta+eps)
            t2 = tf.math.lgamma(theta+eps)
            t3 = tf.math.lgamma(y_true+1.0)
            t4 = -(theta * (tf.math.log(theta+eps)))
            t5 = -(y_true * (tf.math.log(y_pred+eps)))
            t6 = (theta+y_true) * tf.math.log(theta+y_pred+eps)

            final = t1 + t2 + t3 + t4 + t5 + t6

            if reduce:
                final = tf.reduce_mean(final)

        return final

In [8]:
def build_poisson_model(input_shape):
    inputs = Input(
        shape=(input_shape[-1],), name='main_input'
    )

    x = Dense(10)(inputs)
    l = Dense(1, activation=K.exp)(x)
    model = Model(inputs=inputs, outputs=l)
    model.compile(
        loss="poisson",
        optimizer="adam"
    )
    return model

In [9]:
def build_nb_model(input_shape):
    inputs = Input(
        shape=(input_shape[-1],), name='main_input'
    )

    x = Dense(10)(inputs)
    m = Dense(1, activation=K.exp)(x)
    d = Dense(1, activation=lambda x: 1.0/(K.exp(x)+1e-10))(x)
    outputs = SliceLayer(index=0, name='slice')([m, d])
    nb = NB(d)
    model = Model(inputs=inputs, outputs=outputs)
    beta_model = Model(inputs=inputs, outputs=d)
    model.compile(
        loss=nb.loss,
        optimizer="adam"
    )
    return model, beta_model

In [10]:
def generate_data(nb_classes, nb_features, nb_samples_per_class):
    X = []
    y = []
    for cl in range(1, nb_classes + 1):
        for i in range(nb_samples_per_class):
            desired_elements = [1.0] * cl
            undesired_elements = (np.random.random_sample(
                (nb_features - cl)
            ) * 0.1).tolist()
            ll = desired_elements + undesired_elements
            ll = np.array(ll)
            X.append(ll)
            y.append(cl)
    X = np.array(X)
    y = np.array(y)

    return X, y

In [13]:
if __name__ == '__main__':
    os.environ['KERAS_BACKEND'] = 'tensorflow'
    maxcount = 10
    nb_samples = 500
    nb_features = 10
    nb_epochs = 10

    X_train, y_train = generate_data(
        maxcount,
        nb_features,
        nb_samples,
    )

    X_test, y_test = generate_data(
        maxcount,
        nb_features,
        100,
    )
    print('X shape:', X_train.shape, 'type:', X_train.dtype)
    print('y shape:', y_train.shape, 'type:', y_train.dtype)

    # build poisson model
    p_model = build_poisson_model(X_train.shape)

    # fit poisson model
    p_model.fit(
        X_train, y_train,
        epochs=nb_epochs,
        shuffle=True
    )

    # infer mean
    l = p_model.predict(X_test)
    # count inferene
    y_pred_poisson = np.floor(l).astype(np.int)

    # build nb model
    nb_model, beta_model = build_nb_model(X_train.shape)

    # fit negative binomial model
    nb_model.fit(
        X_train, y_train,
        epochs=nb_epochs,
        shuffle=True
    )

    # predict mean
    m = nb_model.predict(
        X_test,
    )
    # predict dispersion
    d = beta_model.predict(X_test)

    # count inference
    y_pred_nb = np.floor(m*((d-1)/d)).astype(np.int)

    f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
    ind = np.arange(y_test.shape[0])
    ax1.pcolormesh(X_test.T)
    ax1.set_xlabel('Input')
    ax2.plot(ind, y_pred_poisson, c='b', alpha=0.7, label='Poisson Count')
    ax2.plot(l, c='m', label='$\lambda$')
    # ax2.plot(ind, y_pred_nb, c='r', alpha=0.7, label='NB')
    ax2.plot(ind, y_test, c='g', label='Ground Truth')
    ax2.set_xlabel('Output')
    plt.legend()
    plt.show()

X shape: (5000, 10) type: float64
y shape: (5000,) type: int64
Epoch 1/10
157/157 [==============================] - 1s 1ms/step - loss: -2.2225
Epoch 2/10
157/157 [==============================] - 0s 2ms/step - loss: -4.6191
Epoch 3/10
157/157 [==============================] - 0s 1ms/step - loss: -4.6495
Epoch 4/10
157/157 [==============================] - 0s 2ms/step - loss: -4.6688
Epoch 5/10
157/157 [==============================] - 0s 2ms/step - loss: -4.6804
Epoch 6/10
157/157 [==============================] - 0s 2ms/step - loss: -4.6877
Epoch 7/10
157/157 [==============================] - 0s 1ms/step - loss: -4.6924
Epoch 8/10
157/157 [==============================] - 0s 1ms/step - loss: -4.6956
Epoch 9/10
157/157 [==============================] - 0s 1ms/step - loss: -4.6978
Epoch 10/10
157/157 [==============================] - 0s 2ms/step - loss: -4.6995
Epoch 1/10


/var/folders/90/98s1yvbs2zq9h146qcbc3t_h0000gn/T/ipykernel_74873/1682128123.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_pred_poisson = np.floor(l).astype(np.int)


TypeError: in user code:

    File "/Library/anaconda3/envs/ds_env/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/Library/anaconda3/envs/ds_env/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/anaconda3/envs/ds_env/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/Library/anaconda3/envs/ds_env/lib/python3.9/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Library/anaconda3/envs/ds_env/lib/python3.9/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/Library/anaconda3/envs/ds_env/lib/python3.9/site-packages/keras/engine/compile_utils.py